In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from scipy import misc
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import facenet
import detect_face
import os
from os.path import join as pjoin
import sys
import time
import copy
import math
import pickle
from sklearn.svm import SVC
from sklearn.externals import joblib
import pymongo
from pymongo import MongoClient
import datetime
from keras.models import load_model
import keras
# import recommender



/Users/ishwarsawale/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

def recom():
    client = MongoClient()
    client = MongoClient('localhost', 27017)
    db = client.retail_db
    print('Creating networks and loading parameters')
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = detect_face.create_mtcnn(sess, './d_npy')

            minsize = 20  # minimum size of face
            threshold = [0.6, 0.7, 0.7]  # three steps's threshold
            factor = 0.709  # scale factor
            margin = 44
            frame_interval = 3
            batch_size = 1000
            image_size = 182
            input_image_size = 160

            HumanNames = os.listdir("./input_dir")
            HumanNames.sort()
            print('Loading feature extraction model')
            modeldir = './pre_model/20170511-185253.pb'
            facenet.load_model(modeldir)

            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]

#             classifier_filename = './my_class/my_classifier.pkl'
#             classifier_filename_exp = os.path.expanduser(classifier_filename)
#             with open(classifier_filename_exp, 'rb') as infile:
#                 (model, class_names) = pickle.load(infile)
#                 print('load classifier file-> %s' % classifier_filename_exp)
            model = load_model('my_model.h5')
            video_capture = cv2.VideoCapture(0)
            c = 0
            counter = 1
            # #video writer
            fourcc = cv2.VideoWriter_fourcc(*'DIVX')
            out = cv2.VideoWriter('3F_0726.avi', fourcc, fps=14, frameSize=(640,480))

            print('Start Recognition!')
            prevTime = 0
            while True:
                ret, frame = video_capture.read()

                frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)    #resize frame (optional)

                curTime = time.time()+1    # calc fps
                timeF = frame_interval
                counter += 1
                if (counter % 12 == 0):
                    if (c % timeF == 0):
                        find_results = []

                        if frame.ndim == 2:
                            frame = facenet.to_rgb(frame)
                        frame = frame[:, :, 0:3]
                        bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
                        nrof_faces = bounding_boxes.shape[0]
                        print('Detected_FaceNum: %d' % nrof_faces)

                        if nrof_faces > 0:
                            det = bounding_boxes[:, 0:4]
                            img_size = np.asarray(frame.shape)[0:2]

                            cropped = []
                            scaled = []
                            scaled_reshape = []
                            bb = np.zeros((nrof_faces,4), dtype=np.int32)

                            for i in range(nrof_faces):
                                emb_array = np.zeros((1, embedding_size))

                                bb[i][0] = det[i][0]
                                bb[i][1] = det[i][1]
                                bb[i][2] = det[i][2]
                                bb[i][3] = det[i][3]

                                # inner exception
                                if bb[i][0] <= 0 or bb[i][1] <= 0 or bb[i][2] >= len(frame[0]) or bb[i][3] >= len(frame):
                                    print('face is inner of range!')
                                    continue

                                cropped.append(frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :])
                                try:
                                    cropped[i] = facenet.flip(cropped[i], False)
                                except Exception as e:
                                    continue
                                scaled.append(misc.imresize(cropped[i], (image_size, image_size), interp='bilinear'))
                                scaled[i] = cv2.resize(scaled[i], (input_image_size,input_image_size),
                                                       interpolation=cv2.INTER_CUBIC)
                                scaled[i] = facenet.prewhiten(scaled[i])
                                scaled_reshape.append(scaled[i].reshape(-1,input_image_size,input_image_size,3))
                                feed_dict = {images_placeholder: scaled_reshape[i], phase_train_placeholder: False}
                                emb_array[0, :] = sess.run(embeddings, feed_dict=feed_dict)
                                predictions = model.predict_proba(emb_array)
                                # print(predictions)
                                best_class_indices = np.argmax(predictions, axis=1)
                                print (predictions)
                                # print(best_class_indices)
                                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                                print('probability score: ', best_class_probabilities)
                                cv2.rectangle(frame, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2)    #boxing face

                                #plot result idx under box
                                text_x = bb[i][0]
                                text_y = bb[i][3] + 20
                                # print('result: ', best_class_indices[0])
                                # print(best_class_indices)
                                # print(HumanNames)
                                for H_i in HumanNames:
                                    # print(H_i)
                                    if best_class_probabilities * 100 > 10:
                                        if HumanNames[best_class_indices[0]] == H_i:
                                            user = {"author": "Admin",
                                                    "text": HumanNames[best_class_indices[0]],
                                                     "date": datetime.datetime.utcnow()}
                                            users = db.users
                                            user_id = users.insert_one(user).inserted_id
                                            result_names = HumanNames[best_class_indices[0]]
                                            print ('recognized user is:', result_names)
                                            cv2.putText(frame, result_names, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                        1, (0, 0, 255), thickness=1, lineType=2)
                                            # recommender.get_prediction(result_names)
                                    else:
                                        result_names = 'Not known'
                                        cv2.putText(frame, result_names, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                        1, (0, 0, 255), thickness=1, lineType=2)
                            else:
                                print('\n')

                        sec = curTime - prevTime
                        prevTime = curTime
                        fps = 1 / (sec)
                        str = 'target'
                        text_fps_x = len(frame[0]) - 150
                        text_fps_y = 20
                        cv2.putText(frame, str, (text_fps_x, text_fps_y),
                                    cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 0), thickness=1, lineType=2)
                        # c+=1
                        cv2.imshow('Video', frame)

                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break

            video_capture.release()
            # #video writer
            out.release()
            cv2.destroyAllWindows()

In [3]:
recom()

Creating networks and loading parameters
Loading feature extraction model
Model filename: ./pre_model/20170511-185253.pb
Start Recognition!
Detected_FaceNum: 1


/Users/ishwarsawale/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:94: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[[1.0000000e+00 8.5281056e-16 6.3200210e-09]]
probability score:  [1.]
recognized user is: Ishwar Sawale


Detected_FaceNum: 1
[[1.0000000e+00 6.9994016e-13 9.0411172e-14]]
probability score:  [1.]
recognized user is: Ishwar Sawale


Detected_FaceNum: 1
[[1.0000000e+00 8.0359080e-14 2.0126418e-14]]
probability score:  [1.]
recognized user is: Ishwar Sawale


Detected_FaceNum: 1
[[1.0000000e+00 1.0615024e-14 1.5293158e-14]]
probability score:  [1.]
recognized user is: Ishwar Sawale


Detected_FaceNum: 1
[[1.0000000e+00 1.3586900e-14 1.7718132e-13]]
probability score:  [1.]
recognized user is: Ishwar Sawale


Detected_FaceNum: 1
[[9.9972540e-01 1.7502928e-13 2.7453134e-04]]
probability score:  [0.9997254]
recognized user is: Ishwar Sawale


Detected_FaceNum: 1
[[1.0000000e+00 3.4746228e-10 1.3194830e-09]]
probability score:  [1.]
recognized user is: Ishwar Sawale


Detected_FaceNum: 0
Detected_FaceNum: 1
[[1.0000000e+00 2.1624602e-10 1.9203628e-10]]
probability score:  [1.]
recognized us